In [1]:
pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=ee27d9e013c665bc4863b2c623fa58c2605e06d5091ad2e97fbcd56d490e083d
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta


In [2]:
pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218910 sha256=4204b8c40284cb4ba835ce105a9219ab5c162009e9906d3567882bb424ee9fd0
  Stored in directory: /root/.cache/pip/wheels/7f/33/8b/50b245c5c65433cd8f5cb24ac15d97e5a3db2d41a8b6ae957d
Successfully built pandas_ta


In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
import ta  # Technical Analysis library
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Bidirectional, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from datetime import datetime, timedelta
import ta
import time
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Bidirectional, LSTM, Dropout, GRU, Dense, Input
import tensorflow as tf
import csv
#################################################################################
currentDate = '2024-12-2'
timeFrame = 5 #in Years
SEQ_LENGTH = 300  # this is the window size
number_of_tickers = 240
#################################################################################
stock_number = 1
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

# Use pandas to read the table from the page
smp500_table = pd.read_html(url)[0]

# Get the first 50 tickers from the table
tickers = smp500_table['Symbol'].head(number_of_tickers).tolist()

results = []

current_date_obj = datetime.strptime(currentDate, '%Y-%m-%d')
start_date_obj = current_date_obj - timedelta(days=timeFrame*365)
start_date = start_date_obj.strftime('%Y-%m-%d')

start_time = time.time()

for ticker in tickers:

  try:

    if stock_number != 1:
      avg_time_per_iteration = elapsed_time / (stock_number - 1)  # average time per iteration so far
      remaining_iterations = number_of_tickers - (stock_number - 1)
      estimated_time_left = avg_time_per_iteration * remaining_iterations

      # Format estimated time left in seconds, minutes, and hours
      estimated_minutes = estimated_time_left / 60
      estimated_seconds = estimated_time_left % 60
      print(f"\nEstimated time left: {estimated_minutes:.2f} minutes")


    stock_data = yf.download(ticker, start=start_date, end=currentDate)

    if stock_data.empty:
          print(f"No data found for {ticker} which is stock # {stock_number} of {number_of_tickers}.")
          stock_number +=1
          continue  # Skip the rest of the loop for this ticker

    # Rest of the processing for the current ticker
    print(f"\nProcessing data for {ticker} which is stock # {stock_number} of {number_of_tickers}.")
    stock_number +=1

    # Assuming stock_data has a 'Date' index
    stock_data['Day_of_Week'] = stock_data.index.dayofweek  # 0=Monday, 6=Sunday
    stock_data['Month'] = stock_data.index.month  # 1 to 12

    # Day of the Week encoding (7 days)
    stock_data['sin_day_of_week'] = np.sin(2 * np.pi * stock_data['Day_of_Week'] / 7)
    stock_data['cos_day_of_week'] = np.cos(2 * np.pi * stock_data['Day_of_Week'] / 7)

    # Month encoding (12 months)
    stock_data['sin_month'] = np.sin(2 * np.pi * stock_data['Month'] / 12)
    stock_data['cos_month'] = np.cos(2 * np.pi * stock_data['Month'] / 12)

    # Time of the Year (Quarter encoding)
    stock_data['Quarter'] = stock_data.index.quarter  # 1 to 4 (Q1, Q2, Q3, Q4)
    stock_data['sin_quarter'] = np.sin(2 * np.pi * stock_data['Quarter'] / 4)
    stock_data['cos_quarter'] = np.cos(2 * np.pi * stock_data['Quarter'] / 4)

    # Drop the original Day_of_Week, Month, and Quarter columns if not needed
    stock_data = stock_data.drop(['Day_of_Week', 'Month', 'Quarter'], axis=1)
    # Feature Engineering: Calculate Technical Indicators
    # Moving Averages
    stock_data['SMA_10'] = stock_data['Close'].rolling(window=10).mean()
    stock_data['SMA_50'] = stock_data['Close'].rolling(window=50).mean()

    # Exponential Moving Averages (EMA)
    stock_data['EMA_10'] = stock_data['Close'].ewm(span=10, adjust=False).mean()
    stock_data['EMA_50'] = stock_data['Close'].ewm(span=50, adjust=False).mean()

    # Bollinger Bands
    stock_data['BB_high'] = stock_data['Close'].rolling(window=20).mean() + 2 * stock_data['Close'].rolling(window=20).std() #SMA with high and low bands based on Standard Diviation
    stock_data['BB_low'] = stock_data['Close'].rolling(window=20).mean() - 2 * stock_data['Close'].rolling(window=20).std()

    # Relative Strength Index (RSI) for overbought or oversold
    stock_data['RSI_14'] = ta.momentum.RSIIndicator(stock_data['Close'].squeeze(), window=14).rsi() #momentum oscilator for speed and change of price movements

    # Moving Average Convergence Divergence (MACD) to find buy and sell signals
    macd_indicator = ta.trend.MACD(stock_data['Close'].squeeze())  # Ensure it's a 1D Series
    stock_data['MACD'] = macd_indicator.macd()
    stock_data['MACD_signal'] = macd_indicator.macd_signal()
    # Create Lag Features (previous 5 days close prices)
    for lag in range(1, 6):
        stock_data[f'Close_lag_{lag}'] = stock_data['Close'].shift(lag)
    # Calculate if the price went up or down compared to the previous day
    stock_data['Price_Change'] = stock_data['Close'].diff()  # Calculate the difference between today's and yesterday's closing price
    stock_data['Target'] = np.where(stock_data['Price_Change'] > 0, 1, 0)  # 1 if price went up, 0 if price went down

    # Drop NaN values created by the .diff() method
    stock_data = stock_data.dropna()
    # Normalize the new features
    scaler = MinMaxScaler()
    try:
      scaled_data = scaler.fit_transform(stock_data)
    except ValueError as e:
      print(f"Scaling failed for {ticker}: {e}")
      continue  # Skip to the next ticker
    # Prepare sliding window sequences for LSTM input
    def create_sequences(data, target, seq_length):
        X, y = [], []
        for i in range(len(data) - seq_length):
            X.append(data[i:i+seq_length, :])  # Use past seq_length days of data
            y.append(target[i+seq_length])  # Predict the target (1 or 0)
        return np.array(X), np.array(y)
    X, y = create_sequences(scaled_data, stock_data['Target'].values, SEQ_LENGTH)
    # Split into training and testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    # Build the model for binary classification


    model = Sequential([
        Input(shape=(X_train.shape[1], X_train.shape[2])),  # Define input shape here
        Bidirectional(LSTM(50, return_sequences=True)),
        Dropout(0.2),
        GRU(50, return_sequences=False),
        Dropout(0.2),
        Dense(25, kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        Dense(1, activation='sigmoid')  # Sigmoid activation for binary classification
    ])
    # Compile the model with binary cross-entropy loss
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    # Early stopping to prevent overfitting
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    # Store the loss for all folds
    train_loss_per_fold = []
    val_loss_per_fold = []

    # Train the model with walk-forward validation
    kf = KFold(n_splits=5, shuffle=False)
    for train_idx, val_idx in kf.split(X):
        X_train_fold, X_val_fold = X[train_idx], X[val_idx]
        y_train_fold, y_val_fold = y[train_idx], y[val_idx]

        # Train the model
        history = model.fit(X_train_fold, y_train_fold,
                        validation_data=(X_val_fold, y_val_fold),
                        epochs=200,  # Increase the number of epochs
                        batch_size=32,
                        callbacks=[early_stopping],
                        verbose=0)
    # Evaluate model on the test data
    test_predictions2 = model.predict(X_test)
    test_predictions2 = (test_predictions2 > 0.5).astype(int)  # Convert predictions to 0 or 1

    # Calculate accuracy
    test_accuracy2 = np.mean(test_predictions2.flatten() == y_test)
    print(f"Accuracy on test data: {test_accuracy2:.4f}")
    predictions = model.predict(X_test)

    # Convert probabilities to binary predictions, but only for confident predictions
    up_predictions = (predictions > 0.5).astype(int)  # Convert to binary class predictions
    up_probabilities = predictions.flatten()

    # Get the model's prediction (probability)
    latest_data = scaled_data[-SEQ_LENGTH:].reshape(1, SEQ_LENGTH, scaled_data.shape[1])
    probability = model.predict(latest_data)[0][0]  # Model returns a 2D array, extract the scalar value
    print(f"Confidence that {ticker} will go up tomorrow: {probability * 100:.1f}%")

    # Print the confidence levelprint(f"Confidence that {ticker} will go up tomorrow: {probability * 100:.2f}%")

    def accuracy(confidence_threshold):

      # Get model's confidence (probabilities)
      predictions = model.predict(X_test)

      # Convert probabilities to binary predictions, but only for confident predictions

      confident_predictions = (predictions >= 0.52).astype(int)  # Convert to binary class predictions
      confident_probabilities = predictions.flatten()

      # Only keep predictions where the model is confident (above the threshold)
      confident_mask = (confident_probabilities > confidence_threshold)  # Filter predictions with high confidence

    # Filtered predictions and true labels based on confidence
      filtered_predictions = confident_predictions[confident_mask]
      filtered_y_test = y_test[confident_mask]
      num_predictions = filtered_predictions.shape[0]

    # If there are no confident predictions, handle this case gracefully
      if len(filtered_predictions) == 0:
          print("No 'up' predictions found!")
          confident_accuracy = 0.000001  # Set to 0 or some other default value
      else:
        # Calculate accuracy based on confident predictions
          confident_accuracy = np.mean(filtered_predictions == filtered_y_test)
          return confident_accuracy
    adjusted_probability1 = accuracy(probability) * 100
    print(f"Adjusted Probability that {ticker} will go up tomorrow: {adjusted_probability1:.1f}%")

    # Get the last close price
    last_close_price = stock_data['Close'].iloc[-1]

    # Placeholder values for other columns
    test_accuracy = test_accuracy2
    test_accuracy2 = f"{test_accuracy:.4f}"
    confidence = f"{probability * 100:.1f}%"
    adjusted_probability = f"{adjusted_probability1:.1f}%"
    # Append row data to results list
    results.append({
        "Ticker": ticker,
        "Price": last_close_price,
        "Test Accuracy": test_accuracy2,
        "Confidence": confidence,
        "Adjusted Probability": adjusted_probability
   })
    elapsed_time = time.time() - start_time

  except Exception as e:
        # Catch any exception and print the error
        print(f"An error occurred with {ticker}: {e}")
        elapsed_time = time.time() - start_time
        continue  # Move to the next iteration

df = pd.DataFrame(results)
# Clean the 'Price' column to extract the numeric value
df['Price'] = df['Price'].astype(float)

# Clean 'Confidence' and 'Adjusted Probability' to numeric values
df['Confidence'] = df['Confidence'].str.rstrip('%').astype(float)
df['Adjusted Probability'] = df['Adjusted Probability'].str.rstrip('%').astype(float)

# Sort by 'Adjusted Probability' and then by 'Confidence', both in descending order
sorted_df = df.sort_values(by=["Confidence", "Adjusted Probability"], ascending=[False, False])
# Optionally, save the sorted DataFrame to a new CSV file
sorted_df.to_csv("sorted_output.csv", index=False)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed



Processing data for MMM which is stock # 1 of 240.


KeyboardInterrupt: 

**CHAPTER TWO!**

In which Pooh decides to make an LSTM model that predicts the price for particular stocks the following day.


In [ ]:
import pandas as pd

# Load the original CSV file
df = pd.read_csv('sorted_output.csv')

# Filter the rows where 'Confidence' is greater than or equal to 0.52
df_filtered = df[df['Confidence'] >= 53]

# Save the filtered DataFrame to a new CSV file
df_filtered.to_csv('sorted_output2.csv', index=False)

print("Filtered data saved to sorted_output2.csv")


Filtered data saved to sorted_output2.csv


**sorted_output2.csv** is the full list of stocks with everything under 52% confidence cut out.

In [ ]:
df = pd.read_csv('sorted_output2.csv')
tickers = df['Ticker'].unique()

In [ ]:

timeFrame = 5 #in Years
SEQ_LENGTH = 300  # this is the window size
number_of_tickers = len(tickers)
#################################################################################
stock_number = 1
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

from keras.callbacks import EarlyStopping, ModelCheckpoint

results = []

current_date_obj = datetime.strptime(currentDate, '%Y-%m-%d')
start_date_obj = current_date_obj - timedelta(days=timeFrame*365)
start_date = start_date_obj.strftime('%Y-%m-%d')

start_time = time.time()

for ticker in tickers:

  try:

    if stock_number != 1:
      avg_time_per_iteration = elapsed_time / (stock_number - 1)  # average time per iteration so far
      remaining_iterations = number_of_tickers - (stock_number - 1)
      estimated_time_left = avg_time_per_iteration * remaining_iterations

      # Format estimated time left in seconds, minutes, and hours
      estimated_minutes = estimated_time_left / 60
      estimated_seconds = estimated_time_left % 60
      print(f"\nEstimated time left: {estimated_minutes:.2f} minutes")


    stock_data = yf.download(ticker, start=start_date, end=currentDate)
    # Select the first level of the MultiIndex (i.e., 'Adj Close', 'Close', etc.)
    stock_data_cleaned = stock_data.xs(key=ticker, axis=1, level=1)
    stock_data = stock_data_cleaned.copy()  # Create a copy of the cleaned data

    if stock_data.empty:
          print(f"No data found for {ticker} which is stock # {stock_number} of {number_of_tickers}.")
          stock_number +=1
          continue  # Skip the rest of the loop for this ticker

    # Rest of the processing for the current ticker
    print(f"\nProcessing data for {ticker} which is stock # {stock_number} of {number_of_tickers}.")
    stock_number +=1

    # Assuming stock_data has a 'Date' index
    stock_data['Day_of_Week'] = stock_data.index.dayofweek  # 0=Monday, 6=Sunday
    stock_data['Month'] = stock_data.index.month  # 1 to 12

    # Day of the Week encoding (7 days)
    stock_data['sin_day_of_week'] = np.sin(2 * np.pi * stock_data['Day_of_Week'] / 7)
    stock_data['cos_day_of_week'] = np.cos(2 * np.pi * stock_data['Day_of_Week'] / 7)

    # Month encoding (12 months)
    stock_data['sin_month'] = np.sin(2 * np.pi * stock_data['Month'] / 12)
    stock_data['cos_month'] = np.cos(2 * np.pi * stock_data['Month'] / 12)

    # Time of the Year (Quarter encoding)
    stock_data['Quarter'] = stock_data.index.quarter  # 1 to 4 (Q1, Q2, Q3, Q4)
    stock_data['sin_quarter'] = np.sin(2 * np.pi * stock_data['Quarter'] / 4)
    stock_data['cos_quarter'] = np.cos(2 * np.pi * stock_data['Quarter'] / 4)

    # Drop the original Day_of_Week, Month, and Quarter columns if not needed
    stock_data = stock_data.drop(['Day_of_Week', 'Month', 'Quarter'], axis=1)
    # Feature Engineering: Calculate Technical Indicators
    # Moving Averages
    stock_data['SMA_10'] = stock_data['Close'].rolling(window=10).mean()
    stock_data['SMA_50'] = stock_data['Close'].rolling(window=50).mean()

    # Exponential Moving Averages (EMA)
    stock_data['EMA_10'] = stock_data['Close'].ewm(span=10, adjust=False).mean()
    stock_data['EMA_50'] = stock_data['Close'].ewm(span=50, adjust=False).mean()

    # Bollinger Bands
    stock_data['BB_high'] = stock_data['Close'].rolling(window=20).mean() + 2 * stock_data['Close'].rolling(window=20).std() #SMA with high and low bands based on Standard Diviation
    stock_data['BB_low'] = stock_data['Close'].rolling(window=20).mean() - 2 * stock_data['Close'].rolling(window=20).std()

    # Relative Strength Index (RSI) for overbought or oversold
    stock_data['RSI_14'] = ta.momentum.RSIIndicator(stock_data['Close'].squeeze(), window=14).rsi() #momentum oscilator for speed and change of price movements

    # Moving Average Convergence Divergence (MACD) to find buy and sell signals
    macd_indicator = ta.trend.MACD(stock_data['Close'].squeeze())  # Ensure it's a 1D Series
    stock_data['MACD'] = macd_indicator.macd()
    stock_data['MACD_signal'] = macd_indicator.macd_signal()
    # Create Lag Features (previous 5 days close prices)
    for lag in range(1, 6):
        stock_data[f'Close_lag_{lag}'] = stock_data['Close'].shift(lag)
    # Step 5: Add daily low and high prices
    stock_data['Low'] = stock_data['Low']  # Already included in the data fetched from Yahoo Finance
    stock_data['High'] = stock_data['High']  # Already included in the data fetched from Yahoo Finance

    # Drop NaN values created by the .diff() method
    stock_data = stock_data.dropna()

    # Normalize the new features
    scaler = MinMaxScaler()
    try:
      scaled_data = scaler.fit_transform(stock_data)
    except ValueError as e:
      print(f"Scaling failed for {ticker}: {e}")
      continue  # Skip to the next ticker

    scaled_data = scaler.fit_transform(stock_data[['Close', 'RSI_14', 'MACD', 'MACD_signal', 'SMA_10', 'SMA_50', 'EMA_10', 'EMA_50',
                                               'BB_high', 'BB_low','Low', 'High', 'Close_lag_1', 'Close_lag_2', 'Close_lag_3', 'Close_lag_4', 'Close_lag_5',
                                               'sin_day_of_week', 'cos_day_of_week', 'sin_month', 'cos_month', 'sin_quarter', 'cos_quarter']])

    # Convert the scaled data back to DataFrame
    scaled_df = pd.DataFrame(scaled_data, columns=['Close', 'RSI_14', 'MACD', 'MACD_signal', 'SMA_10', 'SMA_50', 'EMA_10', 'EMA_50',
                                               'BB_high', 'BB_low','Low', 'High', 'Close_lag_1', 'Close_lag_2', 'Close_lag_3', 'Close_lag_4', 'Close_lag_5',
                                               'sin_day_of_week', 'cos_day_of_week', 'sin_month', 'cos_month', 'sin_quarter', 'cos_quarter'])


    # Step 7: Create Sequences for LSTM
    def create_sequences(data, time_steps=60):
      X, y = [], []
      for i in range(time_steps, len(data)):
            X.append(data[i-time_steps:i, :-1])  # Features: all columns except 'Close'
            y.append(data[i, 0])  # Target: 'Close' price
      return np.array(X), np.array(y)

    X, y = create_sequences(scaled_df.values)

    # Step 8: Split Data into Train and Test Sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

    # Step 9: Define the LSTM Model
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))  # (timesteps, features)
    model.add(LSTM(units=100, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units=100, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=1))  # Output layer for regression

    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

    # Define the callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    model_checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, save_weights_only=False, mode='min')

    # Step 10: Train the model with more epochs and callbacks
    history = model.fit(
       X_train, y_train,
       epochs=100,  # Increase the number of epochs as needed
       batch_size=32,
       validation_data=(X_val, y_val),
       callbacks=[early_stopping, model_checkpoint],
       verbose=0
    )

    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
    import numpy as np

    # Get predictions on the test data
    y_pred = model.predict(X_test)

    # Calculate MAE
    mae = mean_absolute_error(y_test, y_pred)
    print(f'Mean Absolute Error (MAE): {mae}')

    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print(f'Root Mean Squared Error (RMSE): {rmse}')

    # Calculate R² score
    r2 = r2_score(y_test, y_pred)
    print(f'R-squared (R²): {r2}')

    # Step 1: Prepare the last available sequence from the test set (or validation set)
    last_sequence = scaled_df[-SEQ_LENGTH:].values[:, :22]  # Get the last SEQ_LENGTH data points and select the first 22 features

    # Step 2: Make the prediction for the next day
    predicted_stock_price_scaled = model.predict(np.expand_dims(last_sequence, axis=0))  # Expand dims to match input shape

    # Step 3: Inverse scale the prediction
    predicted_stock_price = scaler.inverse_transform(np.column_stack((predicted_stock_price_scaled, np.zeros((predicted_stock_price_scaled.shape[0], scaled_df.shape[1]-1)))))

    # Step 4: Output the predicted price
    predicted_price = predicted_stock_price[0][0]
    print(f"Predicted stock price for tomorrow: ${predicted_price:.2f}")

    # Step 1: Create a baseline prediction (previous day's close)
    y_baseline = np.roll(y_test, shift=1)  # Shift the actual prices by 1 (previous day's close)

    # Step 2: Calculate the baseline error (MAE, RMSE)
    baseline_mae = np.mean(np.abs(y_test - y_baseline))
    baseline_rmse = np.sqrt(np.mean((y_test - y_baseline) ** 2))

    # Step 3: Print Baseline Metrics
    print(f'Baseline Mean Absolute Error (MAE): {baseline_mae}')
    print(f'Baseline Root Mean Squared Error (RMSE): {baseline_rmse}')

    # Optional: R-squared for baseline
    baseline_ss_total = np.sum((y_test - np.mean(y_test)) ** 2)
    baseline_ss_residual = np.sum((y_test - y_baseline) ** 2)
    baseline_r_squared = 1 - (baseline_ss_residual / baseline_ss_total)
    print(f'Baseline R-squared (R²): {baseline_r_squared}')

    results.append({
        "MAE": mae,
        "BaseMAE": baseline_mae,
        "RMSE": rmse,
        "BaseRMSE": baseline_rmse,
        "R2": r2,
        "BaseR2": baseline_r_squared,
        "Ticker": ticker,
        "Predicted Price": predicted_price
    })
    elapsed_time = time.time() - start_time

  except Exception as e:
        # Catch any exception and print the error
        print(f"An error occurred with {ticker}: {e}")
        elapsed_time = time.time() - start_time
        continue  # Move to the next iteration

df2 = pd.DataFrame(results)
# Clean the 'Price' column to extract the numeric value
df2['MAE'] = df2['MAE'].astype(float)
df2['BaseMAE'] = df2['BaseMAE'].astype(float)
df2['RMSE'] = df2['RMSE'].astype(float)
df2['BaseRMSE'] = df2['BaseRMSE'].astype(float)
df2['R2'] = df2['R2'].astype(float)
df2['BaseR2'] = df2['BaseR2'].astype(float)
df2['Ticker'] = df2['Ticker'].astype(str)
df2['Predicted Price'] = df2['Predicted Price'].astype(float)

final_df = pd.merge(df, df2, on='Ticker', how='left')
# Step 4: Save the final DataFrame to sorted_output3.csv
final_df.to_csv('sorted_output3.csv', index=False)

print("sorted_output3.csv has been created with the appended results.")



[*********************100%***********************]  1 of 1 completed


Processing data for AVGO which is stock # 1 of 103.


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Mean Absolute Error (MAE): 0.03580598742391698
Root Mean Squared Error (RMSE): 0.044263839072269
R-squared (R²): 0.8740113337790358
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step


[*********************100%***********************]  1 of 1 completed

Predicted stock price for tomorrow: $173.22
Baseline Mean Absolute Error (MAE): 0.019819579284615314
Baseline Root Mean Squared Error (RMSE): 0.033866061083595356
Baseline R-squared (R²): 0.9262498859084904

Estimated time left: 30.81 minutes

Processing data for HWM which is stock # 2 of 103.


...

Predicted stock price for tomorrow: $34.96
Baseline Mean Absolute Error (MAE): 0.007362893416330218
Baseline Root Mean Squared Error (RMSE): 0.011159235282821426
Baseline R-squared (R²): 0.9483117018937148

Estimated time left: 0.60 minutes

Processing data for APA which is stock # 102 of 103.


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Mean Absolute Error (MAE): 0.01497120780898845
Root Mean Squared Error (RMSE): 0.019504363016581632
R-squared (R²): 0.9279868193354661
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step


[*********************100%***********************]  1 of 1 completed

Predicted stock price for tomorrow: $22.79
Baseline Mean Absolute Error (MAE): 0.011318850921809929
Baseline Root Mean Squared Error (RMSE): 0.024021849041954056
Baseline R-squared (R²): 0.8907651247231062

Estimated time left: 0.30 minutes

Processing data for EXPE which is stock # 103 of 103.


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Mean Absolute Error (MAE): 0.018534765265287686
Root Mean Squared Error (RMSE): 0.026403343867957365
R-squared (R²): 0.9351001502236297
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step
Predicted stock price for tomorrow: $183.47
Baseline Mean Absolute Error (MAE): 0.013450722749328188
Baseline Root Mean Squared Error (RMSE): 0.025491665587204122
Baseline R-squared (R²): 0.9395046138910287
sorted_output3.csv has been created with the appended results.


In **sorted_output3.csv**, we have taken the top 52%+ confidence stocks and added in columns for MAE, RMSE, R2, and **PREDICTED PRICE** from the second LSTM model, as well as the baseline prediction.

In [ ]:
# Filter rows where 'Predicted Price' is greater than or equal to 'Price'
filtered_df = final_df[final_df['Predicted Price'] >= final_df['Price']]

# Save the filtered DataFrame to sorted_output4.csv
filtered_df.to_csv('sorted_output4.csv', index=False, float_format='%.2f')

print("sorted_output4.csv has been created with the filtered rows.")


sorted_output4.csv has been created with the filtered rows.


Our **sorted_output4.csv** file is important.  It is all the stocks with 52%+ probability of going up, but only keeping the stocks that still were predicted to go up with the second LSTM.  So these stocks have passed two AI filters.

**Mess around with the code below.  Check "sorted_output4.csv" and choose your CONFIDENCE_THRESHOLD from there.  Every time you run this, you will get a newly filtered sorted_output5.csv.**

In [ ]:
# Parameters
CONFIDENCE_THRESHOLD = 53  # Minimum AI confidence for trades
ATR_PERIOD = 14  # Number of days for ATR calculation
MIN_VOLUME = 1000000  # Minimum volume to qualify for a trade

# Load the input CSV
df = pd.read_csv("sorted_output4.csv")

# Fetch ATR and Volume data using yfinance
def fetch_atr_and_volume(ticker):
    try:
        stock = yf.Ticker(ticker)
        hist = stock.history(period="3mo")  # Fetch last 3 months of data

        # Calculate ATR
        high_low = hist['High'] - hist['Low']
        high_close = abs(hist['High'] - hist['Close'].shift())
        low_close = abs(hist['Low'] - hist['Close'].shift())
        tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
        atr = tr.rolling(window=ATR_PERIOD).mean().iloc[-1]

        # Get the latest volume
        volume = hist['Volume'].iloc[-1]

        return atr, volume
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return np.nan, np.nan

# Add ATR and Volume columns
df['ATR'], df['Volume'] = zip(*df['Ticker'].apply(fetch_atr_and_volume))

# Filter trades based on conditions
df = df[(df['Predicted Price'] > df['Price']) &  # Predicted price must be higher
        (df['Confidence'] >= CONFIDENCE_THRESHOLD) &  # Confidence threshold
        (df['Volume'] >= MIN_VOLUME)]  # Minimum volume

df.to_csv("sorted_output5.csv", index=False, float_format='%.2f')

print("sorted_output5.csv has been created with the filtered trades.")

sorted_output5.csv has been created with the filtered trades.


**sorted_output5.csv** adds in Volume and ATR.  It also optionally can sort out stocks at a given confidence level.  Remake this several times to weed out stocks before moving on.

In [ ]:
import pandas as pd

# Load the input CSV
df = pd.read_csv("sorted_output5.csv")

# Calculate the new columns
df['Price Change'] = df['Predicted Price'] - df['Price']  # Predicted Price - Price
df['Percent Change'] = (df['Price Change'] / df['Price']) * 100  # (Price Change / Price) * 100
df['Possible Loss'] = ((df['ATR'] / df['Price']) * 100)  # (Price - (Price - ATR)) / Price * 100

# Save the updated DataFrame to a new CSV file
df.to_csv("sorted_output6.csv", index=False, float_format='%.2f')

print("sorted_output6.csv has been created with the new columns.")


sorted_output6.csv has been created with the new columns.


This **sorted_output6.csv** calculates and adds new columns: Price Change, Percent Change, and Possible Loss.

In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('sorted_output6.csv')

# Define a function to calculate Kelly Criterion
def calculate_kelly(row):
    confidence = row['Confidence']
    percent_change = row['Percent Change']
    possible_loss = row['Possible Loss']

    # Calculate Kelly Criterion using the provided formula
    kelly_criterion = (confidence / 100) * (percent_change / 100) / (possible_loss / 100)
    return kelly_criterion

# Apply the function to calculate Kelly Criterion for each row
df['Kelly Criterion'] = df.apply(calculate_kelly, axis=1)

# Save the new dataframe to a new CSV file
df.to_csv('sorted_output7.csv', index=False)

print("New CSV file 'sorted_output7.csv' created with Kelly Criterion column.")


New CSV file 'sorted_output7.csv' created with Kelly Criterion column.


**sorted_output7.csv** calculates the **Kelly Criterion** for each stock.

In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('sorted_output7.csv')

# Select only the desired columns
df_selected = df[['Ticker', 'Price', 'Test Accuracy', 'Confidence', 'Kelly Criterion']]

# Sort the dataframe by Kelly Criterion in descending order
df_sorted = df_selected.sort_values(by='Kelly Criterion', ascending=False)

# Save the new dataframe to a new CSV file
df_sorted.to_csv('sorted_output8.csv', index=False)

print("New CSV file 'sorted_output8.csv' created with selected columns, sorted by Kelly Criterion.")


New CSV file 'sorted_output8.csv' created with selected columns, sorted by Kelly Criterion.


**sorted_output8.csv** removes a lot of columns and then sorts by Kelly Criterion.  Look at this one to manually pick stocks if you would like, or move on to have them picked for you!

In [ ]:
import pandas as pd
from itertools import combinations
import math

# Load the CSV file
df = pd.read_csv('sorted_output8.csv')  # Replace with your actual CSV file path

# Limit to the first 20 tickers
df = df.head(20)

# Set portfolio amount (you can change this value)
portfolio_amount = 40000

# Function to calculate Final Percent for each stock in a combination
def calculate_final_percent(combo, total_kelly_criteria):
    return [(stock['Ticker'], stock['Kelly Criterion'] / total_kelly_criteria * 100) for stock in combo]

# Function to calculate number of shares for a stock (rounding down to nearest whole number)
def calculate_shares(stock, final_percent, portfolio_amount):
    price = stock['Price']
    # Calculate the number of shares and floor the result (round down)
    return math.floor((final_percent * portfolio_amount) // 100 // price)

# Initialize variables for best combination
best_combination = None
best_combination_value = 0
best_combination_stocks = 0

# Check all possible combinations of stocks
for num_stocks in range(1, len(df) + 1):
    for combo in combinations(df.iterrows(), num_stocks):
        # Get the list of stocks in this combination
        selected_stocks = [row[1] for row in combo]

        # Recalculate the total Kelly Criterion sum for this combination
        total_kelly_criteria = sum(stock['Kelly Criterion'] for stock in selected_stocks)

        # Calculate the Final Percent for each stock
        final_percent_list = calculate_final_percent(selected_stocks, total_kelly_criteria)

        # Recalculate Shares based on the Final Percent
        for stock, final_percent in zip(selected_stocks, final_percent_list):
            stock['Final Percent'] = final_percent[1]
            stock['Shares'] = calculate_shares(stock, stock['Final Percent'], portfolio_amount)

        # Filter stocks that can be bought (at least 1 share)
        eligible_stocks = [stock for stock in selected_stocks if stock['Shares'] > 0]

        # Calculate the total cost of this combination
        total_cost = sum(stock['Shares'] * stock['Price'] for stock in eligible_stocks)

        # If the total cost is within the portfolio and better than previous
        if total_cost <= portfolio_amount:
            if len(eligible_stocks) > best_combination_stocks or (len(eligible_stocks) == best_combination_stocks and total_cost > best_combination_value):
                best_combination = eligible_stocks
                best_combination_value = total_cost
                best_combination_stocks = len(eligible_stocks)

# Prepare the output (list of stocks to buy with number of shares)
output = [(stock['Ticker'], stock['Shares']) for stock in best_combination]

# Print the output
print("Best combination of stocks to buy (stock ticker and number of shares):")
for ticker, shares in output:
    print(f"{ticker}: {shares} shares")


Best combination of stocks to buy (stock ticker and number of shares):
AVGO: 42 shares
DXCM: 51 shares
BAX: 96 shares
DOW: 69 shares
CVS: 48 shares
BEN: 123 shares
CVX: 15 shares
APTV: 30 shares
ENPH: 22 shares
COP: 13 shares
CNC: 21 shares
BA: 7 shares
A: 7 shares
SCHW: 12 shares
ABT: 8 shares
AMGN: 3 shares
CTSH: 11 shares
DVN: 23 shares
BBY: 7 shares
ABNB: 4 shares
